### 1. 라이브러리 임포트 및 데이터 불러오기
- 필요한 라이브러리(`pandas`, `torch` 등)를 임포트합니다.
- `ratings.csv` 파일을 읽어옵니다.

In [35]:
import pandas as pd

# 데이터 로드 (경로 확인 필요)
ratings = pd.read_csv('../data/ratings.csv')

# [추가] 연습이니까 데이터 10%만 사용하기 (속도 향상!)
ratings = ratings.sample(frac=0.1, random_state=42) 

print(f"Data size: {len(ratings)}")
ratings.head()

Data size: 3200020


,userId,movieId,rating,timestamp
10685861,66954,781,5.0,850944577
1552723,9877,574,4.0,945495614
6145184,38348,1088,2.0,999974867
16268584,101952,2706,1.0,1203077565
22418634,140400,275079,3.5,1653782463


### 2. 데이터 전처리 (인덱싱)
- `userId`와 `movieId`를 0부터 시작하는 정수 인덱스(`user_idx`, `item_idx`)로 변환합니다.
- 모델의 임베딩 레이어에 입력하기 위해 필요합니다.

In [36]:
user_idx = {u: i for i, u in enumerate(ratings['userId'].unique())}
item_idx = {i: j for j, i in enumerate(ratings['movieId'].unique())}

ratings['user_idx'] = ratings['userId'].map(user_idx)
ratings['item_idx'] = ratings['movieId'].map(item_idx)

num_users = len(user_idx)
num_items = len(item_idx)
print(f"Users: {num_users}, Items: {num_items}")
ratings.head()

Users: 197270, Items: 42809


,userId,movieId,rating,timestamp,user_idx,item_idx
10685861,66954,781,5.0,850944577,0,0
1552723,9877,574,4.0,945495614,1,1
6145184,38348,1088,2.0,999974867,2,2
16268584,101952,2706,1.0,1203077565,3,3
22418634,140400,275079,3.5,1653782463,4,4


### 3. 데이터 나누기 (Train/Val/Test)
- 전체 데이터를 학습용(Train), 검증용(Validation), 테스트용(Test)으로 분리합니다.
- 보통 8:1:1 비율을 사용합니다.

In [37]:
from sklearn.model_selection import train_test_split

# Train/Val/Test 분리 (MF와 동일한 비율)
# 먼저 train+val과 test로 분리 (test: 전체의 10%)
train_val, test = train_test_split(ratings, test_size=0.1, random_state=42)
# train과 validation으로 분리 (전체의 10%가 validation)
# train_val의 약 11.1%가 전체의 10% 정도 됨 (0.9 * 0.111 ≈ 0.1)
train, val = train_test_split(train_val, test_size=0.111, random_state=42)

print(f"Train: {len(train)}, Validation: {len(val)}, Test: {len(test)}")

Train: 2560336, Validation: 319682, Test: 320002


### 4. 데이터셋 클래스 만들기 (Dataset)
- PyTorch `Dataset`을 상속받아 커스텀 데이터셋 클래스를 정의합니다.
- `__getitem__` 메서드에서 사용자, 아이템, 평점을 반환하도록 구현합니다.

In [38]:
import torch
from torch.utils.data import Dataset

class NCFDataset(Dataset):
    def __init__(self, df):
        #데이터프레임에서 필요한 컬럼을 텐서로 변환하여 저장
        self.users = torch.LongTensor(df['user_idx'].values)
        self.items = torch.LongTensor(df['item_idx'].values)
        self.ratings = torch.FloatTensor(df['rating'].values)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]
        
# 데이터셋 객체 생성
train_ds = NCFDataset(train)
val_ds = NCFDataset(val)
test_ds = NCFDataset(test)

### 5. 데이터 로더 만들기 (DataLoader)
- `DataLoader`를 사용하여 데이터를 배치 단위로 불러옵니다.
- 학습 데이터는 섞어줍니다 (`shuffle=True`).

In [39]:
from torch.utils.data import DataLoader

batch_size = 8192

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


### 6. NCF 모델 만들기 (핵심!)
- **GMF (Generalized Matrix Factorization)**: 유저와 아이템 임베딩의 요소별 곱.
- **MLP (Multi-Layer Perceptron)**: 유저와 아이템 임베딩을 연결(Concat)하여 신경망 통과.
- 두 결과를 합쳐서 최종 평점을 예측하는 `NCF` 클래스를 정의합니다.

In [40]:
import torch.nn as nn

class NCF(nn.Module):
    def __init__(self, num_users, num_items, embed_dim=32, mlp_layers=[64, 32, 16]):
        super().__init__()
        
        # 1. GMF (Generalized Matrix Factorization) 파트
        # 유저와 아이템을 각각 임베딩(벡터)으로 바꿈
        self.gmf_user_embedding = nn.Embedding(num_users, embed_dim)
        self.gmf_item_embedding = nn.Embedding(num_items, embed_dim)
        
        # 2. MLP (Multi-Layer Perceptron) 파트
        # MLP용 임베딩은 따로 둠 (GMF랑 다르게 학습되도록)
        self.mlp_user_embedding = nn.Embedding(num_users, embed_dim)
        self.mlp_item_embedding = nn.Embedding(num_items, embed_dim)
        
        # MLP 층 쌓기 (Linear -> ReLU -> Dropout 반복)
        mlp_modules = []
        # 입력 크기: 유저 임베딩 + 아이템 임베딩 (concat하니까 2배)
        input_size = embed_dim * 2 
        for output_size in mlp_layers:
            mlp_modules.append(nn.Linear(input_size, output_size))
            mlp_modules.append(nn.ReLU())
            mlp_modules.append(nn.Dropout(0.2)) # 과적합 방지
            input_size = output_size
        self.mlp_layers = nn.Sequential(*mlp_modules)
        
        # 3. 최종 예측 레이어
        # GMF 출력(embed_dim) + MLP 출력(마지막 층 크기) -> 평점(1개)
        predict_input_size = embed_dim + mlp_layers[-1]
        self.predict_layer = nn.Linear(predict_input_size, 1)
        
        # 가중치 초기화 (처음에 랜덤으로 잘 찍게 도와줌)
        self._init_weights()
        
    def _init_weights(self):
        # 임베딩과 레이어들을 적절한 값으로 초기화
        nn.init.normal_(self.gmf_user_embedding.weight, std=0.01)
        nn.init.normal_(self.gmf_item_embedding.weight, std=0.01)
        nn.init.normal_(self.mlp_user_embedding.weight, std=0.01)
        nn.init.normal_(self.mlp_item_embedding.weight, std=0.01)
        
        for m in self.mlp_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
        
        nn.init.kaiming_uniform_(self.predict_layer.weight, a=1, nonlinearity='sigmoid')

    def forward(self, users, items):
        # 1. GMF 연산: 유저 * 아이템 (요소별 곱)
        gmf_u = self.gmf_user_embedding(users)
        gmf_i = self.gmf_item_embedding(items)
        gmf_out = gmf_u * gmf_i
        
        # 2. MLP 연산: 유저 + 아이템 (이어 붙이기) -> 신경망 통과
        mlp_u = self.mlp_user_embedding(users)
        mlp_i = self.mlp_item_embedding(items)
        mlp_in = torch.cat([mlp_u, mlp_i], dim=1)
        mlp_out = self.mlp_layers(mlp_in)
        
        # 3. 최종 결합: GMF + MLP -> 예측값
        concat = torch.cat([gmf_out, mlp_out], dim=1)
        out = self.predict_layer(concat)
        
        return out.squeeze() # 차원 줄이기 (배치 크기만큼의 1차원 벡터로)

### 7. 학습 준비 (설정)
- 모델, 손실 함수(MSELoss), 최적화 도구(Adam)를 설정합니다.
- GPU(또는 MPS) 사용 설정도 포함합니다.

In [41]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model = NCF(num_users, num_items, embed_dim=32).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Using device: mps


### 8. 학습 루프 (Training Loop)
- 에포크를 돌면서 모델을 학습시킵니다.
- 검증 데이터(Validation)로 성능을 확인하고, 성능이 좋아지면 모델을 저장하는 **Early Stopping**을 구현합니다.

In [42]:
epochs = 10
best_val_rmse = float('inf')
patience = 3
counter = 0
best_model_state = None

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for users, items, batch_ratings in train_loader:
        users, items, batch_ratings = users.to(device), items.to(device), batch_ratings.to(device)

        optimizer.zero_grad() # 지난법 계산 초기화
        outputs = model(users, items) # 예측값
        loss = criterion(outputs, batch_ratings) # 채점
        loss.backward() # 역전파
        optimizer.step() # 최적화
    
        train_loss += loss.item() * users.size(0)
        
    train_rmse = (train_loss / len(train_ds)) ** 0.5


    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for users, items, ratings in val_loader:
            users, items, ratings = users.to(device), items.to(device), ratings.to(device)
            preds = model(users, items)
            val_loss += criterion(preds, ratings).item() * users.size(0)

    val_rmse = (val_loss / len(val_ds)) ** 0.5

    print(f"Epoch {epoch+1:2d} | Train RMSE: {train_rmse:.4f} | Val RMSE: {val_rmse:.4f}")

    # 3. Early Stopping (너무 많이 공부하면 오히려 독이 됨)
    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_model_state = model.state_dict().copy()
        counter = 0
        print("  -> New Best Model! (저장됨)")
    else:
        counter += 1
        print(f"  -> No improvement ({counter}/{patience})")
        if counter >= patience:
            print("Early Stopping! (학습 종료)")
            model.load_state_dict(best_model_state) # 제일 잘했던 때로 되돌리기
            break

Epoch  1 | Train RMSE: 1.8061 | Val RMSE: 0.9119
  -> New Best Model! (저장됨)
Epoch  2 | Train RMSE: 1.0705 | Val RMSE: 0.8925
  -> New Best Model! (저장됨)
Epoch  3 | Train RMSE: 0.9642 | Val RMSE: 0.8838
  -> New Best Model! (저장됨)
Epoch  4 | Train RMSE: 0.8374 | Val RMSE: 0.9042
  -> No improvement (1/3)
Epoch  5 | Train RMSE: 0.7519 | Val RMSE: 0.9223
  -> No improvement (2/3)
Epoch  6 | Train RMSE: 0.6951 | Val RMSE: 0.9412
  -> No improvement (3/3)
Early Stopping! (학습 종료)


### 9. 최종 평가
- 학습에 사용하지 않은 테스트 데이터(Test)로 최종 RMSE를 계산합니다.

In [43]:
# [추가] 인덱스를 영화 ID로 변환하기 위한 리스트 생성
item_ids = list(item_idx.keys())

# 1. 테스트해 볼 유저 한 명 고르기 (예: user_idx 0번)
target_user_idx = 0
target_user_tensor = torch.LongTensor([target_user_idx]).to(device)

# 2. 모든 영화 목록 가져오기
all_item_idxs = torch.arange(num_items).to(device)
# 유저 ID를 영화 개수만큼 복사 (모든 영화에 대해 이 유저가 몇 점 줄지 물어봐야 하니까)
user_input = target_user_tensor.repeat(num_items)

# 3. 모델에게 물어보기 (생각/예측)
model.eval()
with torch.no_grad():
    # "이 유저가 1번 영화는 몇 점? 2번은? 3번은? ..." 하고 싹 다 물어봄
    predicted_ratings = model(user_input, all_item_idxs)

# 4. 점수 높은 순서대로 정렬 (Top 10)
top_ratings, top_indices = torch.topk(predicted_ratings, 10)

# 5. 결과 출력
print(f"User {target_user_idx}에게 추천하는 영화 Top 10:")
for rank, (rating, idx) in enumerate(zip(top_ratings, top_indices)):
    # 원래 영화 ID(movieId)로 다시 변환
    original_movie_id = item_ids[idx.item()]
    print(f"{rank+1}위: 영화ID {original_movie_id} (예상 평점: {rating.item():.2f})")

User 0에게 추천하는 영화 Top 10:
1위: 영화ID 296 (예상 평점: 5.42)
2위: 영화ID 1136 (예상 평점: 5.23)
3위: 영화ID 223 (예상 평점: 5.23)
4위: 영화ID 2324 (예상 평점: 5.16)
5위: 영화ID 593 (예상 평점: 5.16)
6위: 영화ID 1357 (예상 평점: 5.16)
7위: 영화ID 337 (예상 평점: 5.13)
8위: 영화ID 1193 (예상 평점: 5.10)
9위: 영화ID 608 (예상 평점: 5.09)
10위: 영화ID 1225 (예상 평점: 5.07)


In [44]:
# 모델 저장
torch.save(model.state_dict(), '../models/ncf_model.pth')
print("NCF Model saved to ../models/ncf_model.pth")

NCF Model saved to ../models/ncf_model.pth
